In [26]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", unit='bohr', verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

# mycc = cc.CCD(mf,frozen=nfrozen)
# mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sat Oct 18 20:30:33 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000

Set gradient conv threshold to 3.16228e-05
init E= -1.57842868183392
  alpha nocc = 2  HOMO = -0.318199135406173  LUMO = 0.0176832694356078
  beta  nocc = 2  HOMO = -0.320051658639454  LUMO = 0.0172098956498082
cycle= 1 E= -2.14224024703706  delta_E= -0.564  |g|= 0.0529  |ddm|= 0.707
  alpha nocc = 2  HOMO = -0.431955188784667  LUMO = 0.0764361933804052
  beta  nocc = 2  HOMO = -0.432891130414726  LUMO = 0.0761280521183534
cycle= 2 E= -2.14592475253172  delta_E= -0.00368  |g|= 0.0145  |ddm|= 0.183
  alpha nocc = 2  HOMO = -0.43033176186244  LUMO = 0.0874400290571024
  beta  nocc = 2  HOMO = -0.430853479650546  LUMO = 0.0872774781571491
cycle= 3 E= -2.14629904855734  delta_E= -0.000374  |g|= 0.00284  |ddm|= 0.0591
  alpha nocc = 2  HOMO = -0.431966118497162  LUMO = 0.0878067467158269
  beta  nocc = 2  HOMO = -0.432279375249898  LUMO = 0.0876945430480872
cycle= 4 E= -2.14631356663264  delta_E= -1.45e-05  |g|= 0.00035  |ddm|= 0.0126
  alpha nocc = 2  HOMO = -0.432113589414588  LUMO = 0.08

-0.06809544048302607

In [29]:
mycc.t2[0]

array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  3.19656854e-02,  1.98416125e-17,
           4.37507082e-03,  7.53951433e-17,  1.17998159e-02],
         [-3.19656854e-02,  0.00000000e+00, -9.48983634e-04,
          -3.42275743e-18,  6.75439645e-03,  2.68618927e-18],
         [-1.9

In [30]:
import numpy as np
from pyscf import lib
from pyscf.lib import logger
from pyscf.cc import ccsd
from pyscf.cc import uccsd


def update_amps(cc, t1, t2, eris):
    # cc = uccsd.UCCSD(mf)
    time0 = logger.process_clock(), logger.perf_counter()
    log = logger.Logger(cc.stdout, cc.verbose)
    t1a, t1b = t1
    t2aa, t2ab, t2bb = t2
    nocca, noccb, nvira, nvirb = t2ab.shape
    mo_ea_o = eris.mo_energy[0][:nocca]
    mo_ea_v = eris.mo_energy[0][nocca:] + cc.level_shift
    mo_eb_o = eris.mo_energy[1][:noccb]
    mo_eb_v = eris.mo_energy[1][noccb:] + cc.level_shift
    fova = eris.focka[:nocca,nocca:]
    fovb = eris.fockb[:noccb,noccb:]

    u1a = np.zeros_like(t1a)
    u1b = np.zeros_like(t1b)
    #:eris_vvvv = ao2mo.restore(1, np.asarray(eris.vvvv), nvirb)
    #:eris_VVVV = ao2mo.restore(1, np.asarray(eris.VVVV), nvirb)
    #:eris_vvVV = _restore(np.asarray(eris.vvVV), nvira, nvirb)
    #:u2aa += lib.einsum('ijef,aebf->ijab', tauaa, eris_vvvv) * .5
    #:u2bb += lib.einsum('ijef,aebf->ijab', taubb, eris_VVVV) * .5
    #:u2ab += lib.einsum('iJeF,aeBF->iJaB', tauab, eris_vvVV)
    tauaa, tauab, taubb = uccsd.make_tau(t2, t1, t1)
    u2aa, u2ab, u2bb = cc._add_vvvv(None, (tauaa,tauab,taubb), eris)
    u2aa *= .5
    u2bb *= .5

    Fooa =  .5 * lib.einsum('me,ie->mi', fova, t1a)
    Foob =  .5 * lib.einsum('me,ie->mi', fovb, t1b)
    Fvva = -.5 * lib.einsum('me,ma->ae', fova, t1a)
    Fvvb = -.5 * lib.einsum('me,ma->ae', fovb, t1b)
    Fooa += eris.focka[:nocca,:nocca] - np.diag(mo_ea_o)
    Foob += eris.fockb[:noccb,:noccb] - np.diag(mo_eb_o)
    Fvva += eris.focka[nocca:,nocca:] - np.diag(mo_ea_v)
    Fvvb += eris.fockb[noccb:,noccb:] - np.diag(mo_eb_v)
    dtype = u2aa.dtype
    wovvo = np.zeros((nocca,nvira,nvira,nocca), dtype=dtype)
    wOVVO = np.zeros((noccb,nvirb,nvirb,noccb), dtype=dtype)
    woVvO = np.zeros((nocca,nvirb,nvira,noccb), dtype=dtype)
    woVVo = np.zeros((nocca,nvirb,nvirb,nocca), dtype=dtype)
    wOvVo = np.zeros((noccb,nvira,nvirb,nocca), dtype=dtype)
    wOvvO = np.zeros((noccb,nvira,nvira,noccb), dtype=dtype)

    mem_now = lib.current_memory()[0]
    max_memory = max(0, cc.max_memory - mem_now - u2aa.size*8e-6)
    if nvira > 0 and nocca > 0:
        blksize = max(ccsd.BLKMIN, int(max_memory*1e6/8/(nvira**3*3+1)))
        for p0,p1 in lib.prange(0, nocca, blksize):
            ovvv = eris.get_ovvv(slice(p0,p1))  # ovvv = eris.ovvv[p0:p1]
            ovvv = ovvv - ovvv.transpose(0,3,2,1)
            Fvva += np.einsum('mf,mfae->ae', t1a[p0:p1], ovvv)
            wovvo[p0:p1] += lib.einsum('jf,mebf->mbej', t1a, ovvv)
            u1a += 0.5*lib.einsum('mief,meaf->ia', t2aa[p0:p1], ovvv)
            u2aa[:,p0:p1] += lib.einsum('ie,mbea->imab', t1a, ovvv.conj())
            tmp1aa = lib.einsum('ijef,mebf->ijmb', tauaa, ovvv)
            u2aa -= lib.einsum('ijmb,ma->ijab', tmp1aa, t1a[p0:p1]*.5)
            ovvv = tmp1aa = None

    if nvirb > 0 and noccb > 0:
        blksize = max(ccsd.BLKMIN, int(max_memory*1e6/8/(nvirb**3*3+1)))
        for p0,p1 in lib.prange(0, noccb, blksize):
            OVVV = eris.get_OVVV(slice(p0,p1))  # OVVV = eris.OVVV[p0:p1]
            OVVV = OVVV - OVVV.transpose(0,3,2,1)
            Fvvb += np.einsum('mf,mfae->ae', t1b[p0:p1], OVVV)
            wOVVO[p0:p1] = lib.einsum('jf,mebf->mbej', t1b, OVVV)
            u1b += 0.5*lib.einsum('MIEF,MEAF->IA', t2bb[p0:p1], OVVV)
            u2bb[:,p0:p1] += lib.einsum('ie,mbea->imab', t1b, OVVV.conj())
            tmp1bb = lib.einsum('ijef,mebf->ijmb', taubb, OVVV)
            u2bb -= lib.einsum('ijmb,ma->ijab', tmp1bb, t1b[p0:p1]*.5)
            OVVV = tmp1bb = None

    if nvirb > 0 and nocca > 0:
        blksize = max(ccsd.BLKMIN, int(max_memory*1e6/8/(nvira*nvirb**2*3+1)))
        for p0,p1 in lib.prange(0, nocca, blksize):
            ovVV = eris.get_ovVV(slice(p0,p1))  # ovVV = eris.ovVV[p0:p1]
            Fvvb += np.einsum('mf,mfAE->AE', t1a[p0:p1], ovVV)
            woVvO[p0:p1] = lib.einsum('JF,meBF->mBeJ', t1b, ovVV)
            woVVo[p0:p1] = lib.einsum('jf,mfBE->mBEj',-t1a, ovVV)
            u1b += lib.einsum('mIeF,meAF->IA', t2ab[p0:p1], ovVV)
            u2ab[p0:p1] += lib.einsum('IE,maEB->mIaB', t1b, ovVV.conj())
            tmp1ab = lib.einsum('iJeF,meBF->iJmB', tauab, ovVV)
            u2ab -= lib.einsum('iJmB,ma->iJaB', tmp1ab, t1a[p0:p1])
            ovVV = tmp1ab = None

    if nvira > 0 and noccb > 0:
        blksize = max(ccsd.BLKMIN, int(max_memory*1e6/8/(nvirb*nvira**2*3+1)))
        for p0,p1 in lib.prange(0, noccb, blksize):
            OVvv = eris.get_OVvv(slice(p0,p1))  # OVvv = eris.OVvv[p0:p1]
            Fvva += np.einsum('MF,MFae->ae', t1b[p0:p1], OVvv)
            wOvVo[p0:p1] = lib.einsum('jf,MEbf->MbEj', t1a, OVvv)
            wOvvO[p0:p1] = lib.einsum('JF,MFbe->MbeJ',-t1b, OVvv)
            u1a += lib.einsum('iMfE,MEaf->ia', t2ab[:,p0:p1], OVvv)
            u2ab[:,p0:p1] += lib.einsum('ie,MBea->iMaB', t1a, OVvv.conj())
            tmp1abba = lib.einsum('iJeF,MFbe->iJbM', tauab, OVvv)
            u2ab -= lib.einsum('iJbM,MA->iJbA', tmp1abba, t1b[p0:p1])
            OVvv = tmp1abba = None

    eris_ovov = np.asarray(eris.ovov)
    eris_ovoo = np.asarray(eris.ovoo)
    Woooo = lib.einsum('je,nemi->mnij', t1a, eris_ovoo)
    Woooo = Woooo - Woooo.transpose(0,1,3,2)
    Woooo += np.asarray(eris.oooo).transpose(0,2,1,3)
    Woooo += lib.einsum('ijef,menf->mnij', tauaa, eris_ovov) * .5
    u2aa += lib.einsum('mnab,mnij->ijab', tauaa, Woooo*.5)
    Woooo = tauaa = None
    ovoo = eris_ovoo - eris_ovoo.transpose(2,1,0,3)
    Fooa += np.einsum('ne,nemi->mi', t1a, ovoo)
    u1a += 0.5*lib.einsum('mnae,meni->ia', t2aa, ovoo)
    wovvo += lib.einsum('nb,nemj->mbej', t1a, ovoo)
    ovoo = eris_ovoo = None

    tilaa = uccsd.make_tau_aa(t2[0], t1a, t1a, fac=0.5)
    ovov = eris_ovov - eris_ovov.transpose(0,3,2,1)
    Fvva -= .5 * lib.einsum('mnaf,menf->ae', tilaa, ovov)
    Fooa += .5 * lib.einsum('inef,menf->mi', tilaa, ovov)
    Fova = np.einsum('nf,menf->me',t1a, ovov)
    u2aa += ovov.conj().transpose(0,2,1,3) * .5
    wovvo -= 0.5*lib.einsum('jnfb,menf->mbej', t2aa, ovov)
    woVvO += 0.5*lib.einsum('nJfB,menf->mBeJ', t2ab, ovov)
    tmpaa = lib.einsum('jf,menf->mnej', t1a, ovov)
    wovvo -= lib.einsum('nb,mnej->mbej', t1a, tmpaa)
    eris_ovov = ovov = tmpaa = tilaa = None

    eris_OVOV = np.asarray(eris.OVOV)
    eris_OVOO = np.asarray(eris.OVOO)
    WOOOO = lib.einsum('je,nemi->mnij', t1b, eris_OVOO)
    WOOOO = WOOOO - WOOOO.transpose(0,1,3,2)
    WOOOO += np.asarray(eris.OOOO).transpose(0,2,1,3)
    WOOOO += lib.einsum('ijef,menf->mnij', taubb, eris_OVOV) * .5
    u2bb += lib.einsum('mnab,mnij->ijab', taubb, WOOOO*.5)
    WOOOO = taubb = None
    OVOO = eris_OVOO - eris_OVOO.transpose(2,1,0,3)
    Foob += np.einsum('ne,nemi->mi', t1b, OVOO)
    u1b += 0.5*lib.einsum('mnae,meni->ia', t2bb, OVOO)
    wOVVO += lib.einsum('nb,nemj->mbej', t1b, OVOO)
    OVOO = eris_OVOO = None

    tilbb = uccsd.make_tau_aa(t2[2], t1b, t1b, fac=0.5)
    OVOV = eris_OVOV - eris_OVOV.transpose(0,3,2,1)
    Fvvb -= .5 * lib.einsum('MNAF,MENF->AE', tilbb, OVOV)
    Foob += .5 * lib.einsum('inef,menf->mi', tilbb, OVOV)
    Fovb = np.einsum('nf,menf->me',t1b, OVOV)
    u2bb += OVOV.conj().transpose(0,2,1,3) * .5
    wOVVO -= 0.5*lib.einsum('jnfb,menf->mbej', t2bb, OVOV)
    wOvVo += 0.5*lib.einsum('jNbF,MENF->MbEj', t2ab, OVOV)
    tmpbb = lib.einsum('jf,menf->mnej', t1b, OVOV)
    wOVVO -= lib.einsum('nb,mnej->mbej', t1b, tmpbb)
    eris_OVOV = OVOV = tmpbb = tilbb = None

    eris_OVoo = np.asarray(eris.OVoo)
    eris_ovOO = np.asarray(eris.ovOO)
    Fooa += np.einsum('NE,NEmi->mi', t1b, eris_OVoo)
    u1a -= lib.einsum('nMaE,MEni->ia', t2ab, eris_OVoo)
    wOvVo -= lib.einsum('nb,MEnj->MbEj', t1a, eris_OVoo)
    woVVo += lib.einsum('NB,NEmj->mBEj', t1b, eris_OVoo)
    Foob += np.einsum('ne,neMI->MI', t1a, eris_ovOO)
    u1b -= lib.einsum('mNeA,meNI->IA', t2ab, eris_ovOO)
    woVvO -= lib.einsum('NB,meNJ->mBeJ', t1b, eris_ovOO)
    wOvvO += lib.einsum('nb,neMJ->MbeJ', t1a, eris_ovOO)
    WoOoO = lib.einsum('JE,NEmi->mNiJ', t1b, eris_OVoo)
    WoOoO+= lib.einsum('je,neMI->nMjI', t1a, eris_ovOO)
    WoOoO += np.asarray(eris.ooOO).transpose(0,2,1,3)
    eris_OVoo = eris_ovOO = None

    eris_ovOV = np.asarray(eris.ovOV)
    WoOoO += lib.einsum('iJeF,meNF->mNiJ', tauab, eris_ovOV)
    u2ab += lib.einsum('mNaB,mNiJ->iJaB', tauab, WoOoO)
    WoOoO = None

    tilab = uccsd.make_tau_ab(t2[1], t1 , t1 , fac=0.5)
    Fvva -= lib.einsum('mNaF,meNF->ae', tilab, eris_ovOV)
    Fvvb -= lib.einsum('nMfA,nfME->AE', tilab, eris_ovOV)
    Fooa += lib.einsum('iNeF,meNF->mi', tilab, eris_ovOV)
    Foob += lib.einsum('nIfE,nfME->MI', tilab, eris_ovOV)
    Fova += np.einsum('NF,meNF->me',t1b, eris_ovOV)
    Fovb += np.einsum('nf,nfME->ME',t1a, eris_ovOV)
    u2ab += eris_ovOV.conj().transpose(0,2,1,3)
    wovvo += 0.5*lib.einsum('jNbF,meNF->mbej', t2ab, eris_ovOV)
    wOVVO += 0.5*lib.einsum('nJfB,nfME->MBEJ', t2ab, eris_ovOV)
    wOvVo -= 0.5*lib.einsum('jnfb,nfME->MbEj', t2aa, eris_ovOV)
    woVvO -= 0.5*lib.einsum('JNFB,meNF->mBeJ', t2bb, eris_ovOV)
    woVVo += 0.5*lib.einsum('jNfB,mfNE->mBEj', t2ab, eris_ovOV)
    wOvvO += 0.5*lib.einsum('nJbF,neMF->MbeJ', t2ab, eris_ovOV)
    tmpabab = lib.einsum('JF,meNF->mNeJ', t1b, eris_ovOV)
    tmpbaba = lib.einsum('jf,nfME->MnEj', t1a, eris_ovOV)
    woVvO -= lib.einsum('NB,mNeJ->mBeJ', t1b, tmpabab)
    wOvVo -= lib.einsum('nb,MnEj->MbEj', t1a, tmpbaba)
    woVVo += lib.einsum('NB,NmEj->mBEj', t1b, tmpbaba)
    wOvvO += lib.einsum('nb,nMeJ->MbeJ', t1a, tmpabab)
    tmpabab = tmpbaba = tilab = None

    Fova += fova
    Fovb += fovb
    u1a += fova.conj()
    u1a += np.einsum('ie,ae->ia', t1a, Fvva)
    u1a -= np.einsum('ma,mi->ia', t1a, Fooa)
    u1a -= np.einsum('imea,me->ia', t2aa, Fova)
    u1a += np.einsum('iMaE,ME->ia', t2ab, Fovb)
    u1b += fovb.conj()
    u1b += np.einsum('ie,ae->ia',t1b,Fvvb)
    u1b -= np.einsum('ma,mi->ia',t1b,Foob)
    u1b -= np.einsum('imea,me->ia', t2bb, Fovb)
    u1b += np.einsum('mIeA,me->IA', t2ab, Fova)

    eris_oovv = np.asarray(eris.oovv)
    eris_ovvo = np.asarray(eris.ovvo)
    wovvo -= eris_oovv.transpose(0,2,3,1)
    wovvo += eris_ovvo.transpose(0,2,1,3)
    oovv = eris_oovv - eris_ovvo.transpose(0,3,2,1)
    u1a-= np.einsum('nf,niaf->ia', t1a,      oovv)
    tmp1aa = lib.einsum('ie,mjbe->mbij', t1a,      oovv)
    u2aa += 2*lib.einsum('ma,mbij->ijab', t1a, tmp1aa)
    eris_ovvo = eris_oovv = oovv = tmp1aa = None

    eris_OOVV = np.asarray(eris.OOVV)
    eris_OVVO = np.asarray(eris.OVVO)
    wOVVO -= eris_OOVV.transpose(0,2,3,1)
    wOVVO += eris_OVVO.transpose(0,2,1,3)
    OOVV = eris_OOVV - eris_OVVO.transpose(0,3,2,1)
    u1b-= np.einsum('nf,niaf->ia', t1b,      OOVV)
    tmp1bb = lib.einsum('ie,mjbe->mbij', t1b,      OOVV)
    u2bb += 2*lib.einsum('ma,mbij->ijab', t1b, tmp1bb)
    eris_OVVO = eris_OOVV = OOVV = None

    eris_ooVV = np.asarray(eris.ooVV)
    eris_ovVO = np.asarray(eris.ovVO)
    woVVo -= eris_ooVV.transpose(0,2,3,1)
    woVvO += eris_ovVO.transpose(0,2,1,3)
    u1b+= np.einsum('nf,nfAI->IA', t1a, eris_ovVO)
    tmp1ab = lib.einsum('ie,meBJ->mBiJ', t1a, eris_ovVO)
    tmp1ab+= lib.einsum('IE,mjBE->mBjI', t1b, eris_ooVV)
    u2ab -= lib.einsum('ma,mBiJ->iJaB', t1a, tmp1ab)
    eris_ooVV = eris_ovVO = tmp1ab = None

    eris_OOvv = np.asarray(eris.OOvv)
    eris_OVvo = np.asarray(eris.OVvo)
    wOvvO -= eris_OOvv.transpose(0,2,3,1)
    wOvVo += eris_OVvo.transpose(0,2,1,3)
    u1a+= np.einsum('NF,NFai->ia', t1b, eris_OVvo)
    tmp1ba = lib.einsum('IE,MEbj->MbIj', t1b, eris_OVvo)
    tmp1ba+= lib.einsum('ie,MJbe->MbJi', t1a, eris_OOvv)
    u2ab -= lib.einsum('MA,MbIj->jIbA', t1b, tmp1ba)
    eris_OOvv = eris_OVvo = tmp1ba = None

    u2aa += 2*lib.einsum('imae,mbej->ijab', t2aa, wovvo)
    u2aa += 2*lib.einsum('iMaE,MbEj->ijab', t2ab, wOvVo)
    u2bb += 2*lib.einsum('imae,mbej->ijab', t2bb, wOVVO)
    u2bb += 2*lib.einsum('mIeA,mBeJ->IJAB', t2ab, woVvO)
    u2ab += lib.einsum('imae,mBeJ->iJaB', t2aa, woVvO)
    u2ab += lib.einsum('iMaE,MBEJ->iJaB', t2ab, wOVVO)
    u2ab += lib.einsum('iMeA,MbeJ->iJbA', t2ab, wOvvO)
    u2ab += lib.einsum('IMAE,MbEj->jIbA', t2bb, wOvVo)
    u2ab += lib.einsum('mIeA,mbej->jIbA', t2ab, wovvo)
    u2ab += lib.einsum('mIaE,mBEj->jIaB', t2ab, woVVo)
    wovvo = wOVVO = woVvO = wOvVo = woVVo = wOvvO = None

    Ftmpa = Fvva - .5*lib.einsum('mb,me->be', t1a, Fova)
    Ftmpb = Fvvb - .5*lib.einsum('mb,me->be', t1b, Fovb)
    u2aa += lib.einsum('ijae,be->ijab', t2aa, Ftmpa)
    u2bb += lib.einsum('ijae,be->ijab', t2bb, Ftmpb)
    u2ab += lib.einsum('iJaE,BE->iJaB', t2ab, Ftmpb)
    u2ab += lib.einsum('iJeA,be->iJbA', t2ab, Ftmpa)
    Ftmpa = Fooa + 0.5*lib.einsum('je,me->mj', t1a, Fova)
    Ftmpb = Foob + 0.5*lib.einsum('je,me->mj', t1b, Fovb)
    u2aa -= lib.einsum('imab,mj->ijab', t2aa, Ftmpa)
    u2bb -= lib.einsum('imab,mj->ijab', t2bb, Ftmpb)
    u2ab -= lib.einsum('iMaB,MJ->iJaB', t2ab, Ftmpb)
    u2ab -= lib.einsum('mIaB,mj->jIaB', t2ab, Ftmpa)

    eris_ovoo = np.asarray(eris.ovoo).conj()
    eris_OVOO = np.asarray(eris.OVOO).conj()
    eris_OVoo = np.asarray(eris.OVoo).conj()
    eris_ovOO = np.asarray(eris.ovOO).conj()
    ovoo = eris_ovoo - eris_ovoo.transpose(2,1,0,3)
    OVOO = eris_OVOO - eris_OVOO.transpose(2,1,0,3)
    u2aa -= lib.einsum('ma,jbim->ijab', t1a, ovoo)
    u2bb -= lib.einsum('ma,jbim->ijab', t1b, OVOO)
    u2ab -= lib.einsum('ma,JBim->iJaB', t1a, eris_OVoo)
    u2ab -= lib.einsum('MA,ibJM->iJbA', t1b, eris_ovOO)
    eris_ovoo = eris_OVoo = eris_OVOO = eris_ovOO = None

    u2aa *= .5
    u2bb *= .5
    u2aa = u2aa - u2aa.transpose(0,1,3,2)
    u2aa = u2aa - u2aa.transpose(1,0,2,3)
    u2bb = u2bb - u2bb.transpose(0,1,3,2)
    u2bb = u2bb - u2bb.transpose(1,0,2,3)

    eia_a = lib.direct_sum('i-a->ia', mo_ea_o, mo_ea_v)
    eia_b = lib.direct_sum('i-a->ia', mo_eb_o, mo_eb_v)
    u1a /= eia_a
    u1b /= eia_b

    u2aa /= lib.direct_sum('ia+jb->ijab', eia_a, eia_a)
    u2ab /= lib.direct_sum('ia+jb->ijab', eia_a, eia_b)
    u2bb /= lib.direct_sum('ia+jb->ijab', eia_b, eia_b)

    time0 = log.timer_debug1('update t1 t2', *time0)
    t1new = 0*u1a, 0*u1b
    t2new = u2aa, u2ab, u2bb
    return t1new, t2new

In [32]:
from pyscf.cc import uccsd
mycc = uccsd.UCCSD(mf,frozen=nfrozen)
uccsd.UCCSD.update_amps = update_amps
mycc.kernel()[0]


******** <class 'pyscf.cc.uccsd.UCCSD'> ********
CC2 = 0
CCSD nocc = (2, 2), nmo = (8, 8)
frozen orbitals 0
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-06
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 189 MB)
Init t2, MP2 energy = -0.044213456283748
Init E_corr(UCCSD) = -0.0442134562838028
cycle = 1  E_corr(UCCSD) = -0.0576747647018351  dE = -0.0134613084  norm(t1,t2) = 0.059159
cycle = 2  E_corr(UCCSD) = -0.0627116909495396  dE = -0.00503692625  norm(t1,t2) = 0.0279152
cycle = 3  E_corr(UCCSD) = -0.0668608855315327  dE = -0.00414919458  norm(t1,t2) = 0.0142028
cycle = 4  E_corr(UCCSD) = -0.0667859122981428  dE = 7.49732334e-05  norm(t1,t2) = 0.00215945
cycle = 5  E_corr(UCCSD) = -0.0668180177018729  dE = -3.21054037e-05  norm(t1,t2) = 0.000433978
cycle = 6  E_corr(UCCSD) = -0.0668209941767414  dE = -2.97647487e-06  norm(t1,t2) = 6.61078e-05
cycle = 7  E_corr(UCCSD) = -0.0668192520667748  dE = 1.74210997e-06  norm

-0.06681997993639967

In [33]:
mycc.t1

(array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]))

In [40]:
print(mycc.t1[0].shape,mycc.t1[1].shape)
mycc.t1 = (mycc.t1[0]*0,mycc.t1[1]*0)

(2, 6) (2, 6)


In [41]:
print(mycc.t1[0].shape,mycc.t1[1].shape)
print(mycc.t1[0],mycc.t1[1])

(2, 6) (2, 6)
[[-0.  0.  0. -0.  0.  0.]
 [-0. -0.  0.  0.  0.  0.]] [[-0. -0.  0. -0.  0.  0.]
 [ 0. -0.  0.  0.  0.  0.]]


In [2]:
print(mol.nelectron//2)
print(mol.nao)

2
8


In [8]:
def thouless_trans(t1):
    ''' thouless transformation |psi'> = exp(t1)|psi>
        gives the transformed mo rep in the 
        original mo basis <psi_p|psi'_i>
    '''
    # t = t_ia
    # t_ia = c_ik c.T_ka
    # c_ik = <psi_i|psi'_k>
    q, r = jnp.linalg.qr(t1,mode='complete')
    u_ji = q
    u_ai = r.T
    u_occ = jnp.vstack((u_ji,u_ai))
    mo_t, _ = jnp.linalg.qr(u_occ,mode='complete')
    # u, _, _ = np.linalg.svd(u_occ)
    return mo_t

In [3]:
t1 = mycc.t1*5
q, r = jnp.linalg.qr(t1,mode='complete')
u_ji = q
u_ai = r.T
u_occ = jnp.vstack((u_ji,u_ai))
mo_t_occ, _ = jnp.linalg.qr(u_occ)
mo_t, _ = jnp.linalg.qr(u_occ,mode='complete')

In [5]:
nocc = mol.nelectron//2
print(mo_t_occ.shape,mo_t.shape)
print(mo_t_occ)
print(mo_t[:,:nocc])

(8, 2) (8, 8)
[[-9.9758232e-01 -1.5199196e-14]
 [ 1.5352736e-14 -9.9415469e-01]
 [ 3.9043330e-02 -3.9447164e-18]
 [-2.6860154e-15 -5.8827437e-02]
 [-5.5959675e-02  2.2853071e-16]
 [ 1.6771570e-15 -7.9744615e-02]
 [-1.3177985e-02 -2.7718166e-15]
 [-1.2296816e-15 -4.2852178e-02]]
[[-9.9758232e-01 -1.5199196e-14]
 [ 1.5352736e-14 -9.9415469e-01]
 [ 3.9043330e-02 -3.9447164e-18]
 [-2.6860154e-15 -5.8827437e-02]
 [-5.5959675e-02  2.2853071e-16]
 [ 1.6771570e-15 -7.9744615e-02]
 [-1.3177985e-02 -2.7718166e-15]
 [-1.2296816e-15 -4.2852178e-02]]


In [ ]:
# import numpy as np
# np.set_printoptions(5)
# # AFQMC/CCSD_PT energy: -1.096171 +/- 0.000554
# # AFQMC/CCSD_PT energy: -2.192221 +/- 0.001636
# # AFQMC/CCSD_PT energy: -4.384578 +/- 0.006862
# # AFQMC/CCSD_PT energy: -8.769366 +/- 0.019849
# # AFQMC/CCSD_PT energy: -17.538377 +/- 0.091299
# # AFQMC/CCSD_PT energy: -27.405295 +/- 0.246627
# nm = np.array([1,2,4,8,16,25])
# ept = np.array([-1.096171,-2.192221,-4.384578,-8.769366,-17.538377,-27.405295])
# ept_perm = ept/nm
# print('number of H2 monomers:            ', nm)
# print('AFQMC/CCSD_PT energy per monomer: ', ept_perm)

number of H2 monomers:             [ 1  2  4  8 16 25]
AFQMC/CCSD_PT energy per monomer:  [-1.09617 -1.09611 -1.09614 -1.09617 -1.09615 -1.09621]


In [9]:
from ad_afqmc.cisd_perturb import ccsd_pt
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 2,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
from ad_afqmc.cisd_perturb import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
mycc.t1 = 5*mycc.t1
ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
# pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (2, 2)
# Number of basis functions: 8
# Number of Cholesky vectors: 26
#


In [10]:
from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

trial = wavefunctions.rhf(
    trial.norb, trial.nelec,n_batch=trial.n_batch
    )

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 8
# nelec: (2, 2)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 2
# seed: 2
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [ ]:
# def thouless_trans(t1):
#     q, r = np.linalg.qr(t1)
#     u_ai = r.T
#     u_ji = q
#     u_occ = np.vstack((u_ji,u_ai))
#     q, r = np.linalg.qr(u_occ)
#     # u, _, _ = np.linalg.svd(u_occ)
#     return q

In [12]:
t1, t2 = wave_data['ci1'], wave_data['ci2']
mo_t = thouless_trans(t1)
nocc = wave_data['ci1'].shape[0]
norb = trial.norb
# trial_mo = u @ np.eye(norb)[:,:nocc]
wave_data['mo_coeff'] = mo_t[:,:nocc]
# print(t1.shape,t2.shape,trial_mo.shape)
# rot_t1 = mo_t.T.conjugate()[:,:nocc] @ t1
rot_t2 = jnp.einsum('il,jk,lakb->iajb',mo_t[:nocc,:nocc].T.conjugate(),
                   mo_t[:nocc,:nocc].T.conjugate(),t2)
wave_data['t1'] = t1
wave_data['t2'] = t2
# wave_data['rot_t1'] = rot_t1
wave_data['rot_t2'] = rot_t2

In [13]:
import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)

ham_data['h1_mod'] = h1_mod

In [17]:
from jax import lax
### <psi|T2(h1+h2)|phi>/<psi|phi> ### 
@partial(jit, static_argnums=2)
def _t2_walker_olp(walker,wave_data,trial):
    ''' t_iajb <psi|ijab|phi> '''
    rot_t2 = wave_data['rot_t2']
    nocc = walker.shape[1]
    # GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    GF = trial._calc_green(walker, wave_data)
    # o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    # <psi|phi>
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    # t_iajb <psi|ijab|phi>/<psi|phi>
    o2 = 2 * jnp.einsum(
        "iajb, ia, jb", rot_t2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", rot_t2, GF[:, nocc:], GF[:, nocc:])
    return o2 * o0

@partial(jit, static_argnums=4)
def _t2_exp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict,trial) -> complex:
    '''
    t_iajb <psi|ijab exp(x*h1_mod)|walker>/<psi|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    olp = _t2_walker_olp(walker_1x,wave_data,trial)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return olp/o0

@partial(jit, static_argnums=4)
def _t2_exp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict, trial) -> complex:
    '''
    t_iajb <psi|ijab exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _t2_walker_olp(walker_2x,wave_data,trial)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return olp/o0

@partial(jit, static_argnums=3)
def _calc_energy_pt2(walker, ham_data, wave_data, trial):
    ''' 
    t = <psi|T2|phi>/<psi|phi>
    e0 = <psi|H|phi>/<psi|phi>
    e1 = <psi|T2(h1+h2)|phi>/<psi|phi>
    '''
    eps=1e-4

    norb = trial.norb
    # h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    # t_ij^ab <psi|ijab|phi_1x>
    x = 0.0
    f1 = lambda a: _t2_exp1(a,h1_mod,walker,wave_data,trial)
    t_exp1_0, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    # t_ij^ab <psi|ijab|phi_2x>
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _t2_exp2(eps,c,walker,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps
   
    # o0 = trial._calc_overlap_restricted(walker, wave_data)
    et2 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )

    # t2 = _t2_walker_olp(walker,wave_data,trial)/o0
    t2 = t_exp1_0
    e0 = trial._calc_energy_restricted(walker,ham_data,wave_data)

    return jnp.real(t2), jnp.real(e0), jnp.real(et2)

In [15]:
from ad_afqmc.cisd_perturb import ccsd_pt2

config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

h0 = ham_data['h0']
t, e0, e1 = ccsd_pt2._ccsd_walker_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
init_ept = e0 + e1 - t*(e0-h0)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init rhf walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
print('init ccsd_pt_walker energy: ', init_ept)
print('ccsd energy: ', mycc.e_tot)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
print('ccsd energy with t1 t2: ', mf.e_tot+eccsd)
eccsd = mycc.energy(mycc.t1, 0*mycc.t2, eris)
print('ccsd energy with t1 0*t2: ', mf.e_tot+eccsd)
eccsd = mycc.energy(mycc.t1*0, mycc.t2, eris)
print('ccsd energy with 0*t1 t2: ', mf.e_tot+eccsd)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init rhf walker energy:  -2.1454872416253337
mf enegry:  -2.146313938492948
init ccsd_pt_walker energy:  -2.2136157627186845
ccsd energy:  -2.2144093895871735
ccsd energy with t1 t2:  -2.213615759902119
ccsd energy with t1 0*t2:  -2.1454872423581426
ccsd energy with 0*t1 t2:  -2.2144424560369242


In [18]:
t, e0, e1 = _calc_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))

-2.2136157591670838


In [18]:
trial2 = wavefunctions.ccsd_pt2_ad(trial.norb, trial.nelec, n_batch=trial.n_batch)
mo_t = trial2.thouless_trans(wave_data['t1'])
wave_data['mo_coeff'] = mo_t
t, e0, e1 = trial2._calc_energy_pt_restricted(
    prop_data['walkers'][0],ham_data,wave_data)
init_ept = e0 + e1 - t*(e0-h0)
print(init_ept)

-2.1652917893645203


In [24]:
from ad_afqmc.cisd_perturb import sample_ccsd_pt2
prop_data["n_killed_walkers"] = 0
prop_data, (blk_wt, blk_t, blk_e0, blk_e1) = \
    sample_ccsd_pt2._block_scan(prop_data,ham_data,prop,trial,wave_data,sampler)
print(blk_wt, blk_t, blk_e0, blk_e1)
print(blk_e0+blk_e1-blk_t*(blk_e0-h0))

1.9983712866932402 0.02324355211854993 -2.17346123974168 -0.1468656262740246
-2.2194468768219293


In [25]:
t, e0, e1 = ccsd_pt2._ccsd_walker_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))
t, e0, e1 = _calc_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))

-2.2034847267664386
-2.2034847360714407
